In [7]:
import numpy as np
import matplotlib.pyplot as plt
import xtrack as xt
import xcoll as xc
import apertls

In [3]:
xt.RBend?

Init signature:
xt.RBend(
    order=None,
    knl: List[float] = None,
    ksl: List[float] = None,
    **kwargs,
)
Docstring:     
Implementation of a straight combined function magnet (i.e. a rectangular
bending magnet with a quadrupole component).

Parameters
----------
k0 : float, optional
    Strength of the horizontal dipolar component in units of m^-1.
k1 : float, optional
    Strength of the horizontal quadrupolar component in units of m^-2.
h : float, optional
    Curvature of the reference trajectory in units of m^-1. Will be
    computed from angle and `length_straight` if not given, otherwise will
    be checked for consistency. Changes to `h` will update `angle` and
    `length`.
angle : float, optional
    Angle of the bend in radians. Will be computed from `h` and
    `length_straight` if not given, otherwise will be checked for
    consistency. Changes to `angle` will update `h` and `length`.
k0_from_h : bool, optional
    If True, `k0` will assume the value of `h` and 

In [4]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta5.json')

Loading line from dict:   0%|          | 0/36404 [00:00<?, ?it/s]

Done loading line from dict.           


In [9]:
tt = line.get_table()
tw = line.twiss()

In [10]:
tt.rows['mb.*']

Table: 18600 rows, 11 cols
name                             s element_type        isthick isreplica parent_name iscollective ...
mba.10030_entry            3.44499 Marker                False     False None               False
mba.10030..entry_map       3.44499 ThinSliceRBendEntry   False     False mba.10030          False
mba.10030..0_aper2         3.44499 LimitRect             False     False None               False
mba.10030..0               3.44499 ThickSliceRBend        True     False mba.10030          False
mba.10030..1_aper2         4.07099 LimitRect             False     False None               False
mba.10030..1               4.07099 ThickSliceRBend        True     False mba.10030          False
mba.10030..2_aper2         4.69699 LimitRect             False     False None               False
mba.10030..2               4.69699 ThickSliceRBend        True     False mba.10030          False
mba.10030..3_aper2           5.323 LimitRect             False     False None          

In [11]:
line['mba.10030..0']

View of ThickSliceRBend(parent_name='mba.10030', _parent=RBend(length_straight=6.26, angle=0.00845, length=6.26, k0=0.00135, k1=0, h=0.00135, k0_from_h=True, model='adaptive', knl=array([0., 0., 0., 0., 0., 0.]), ksl=array([0., 0., 0., 0., 0., 0.]), edge_entry_active=np.int64(1), edge_exit_active=np.int64(1), edge_entry_model='linear', edge_exit_model='linear', edge_entry_angle=0, edge_exit_angle=0, edge_entry_angle_fdown=0, edge_exit_angle_fdown=0, edge_entry_fint=0, edge_exit_fint=0, edge_entry_hgap=0, edge_exit_hgap=0, shift_x=0, shift_y=0, rot_s_rad=0), radiation_flag=np.int64(10), delta_taper=0, weight=0.1)

In [15]:
rbend = line['mba.10030..0']._parent.copy()
aperture = line['mba.10030..0_aper2'].copy()

In [27]:
rbend.k0

np.float64(0.0013490601351007177)

In [32]:
env2 = xt.Environment()
line2 = env2.new_line(components=[env2.new('aper_entry', xt.LimitRect, min_x=-0.0716, max_x=0.0804, min_y=-0.0193, max_y=0.0193),
                                  env2.new('rbend', xt.RBend, length_straight=6.26, angle=0.008445141542, length=6.260018602780449, k0=0.0013490601351007177, k1=0, h=0.0013490601351007177, k0_from_h=True),
                                  env2.new('aper_exit', xt.LimitRect, min_x=-0.0716, max_x=0.0804, min_y=-0.0193, max_y=0.0193)])
line2.particle_ref = line.particle_ref.copy()

In [34]:
# tw = line2.twiss()
tt = line2.get_table()

In [35]:
tt

Table: 4 rows, 11 cols
name                   s element_type isthick isreplica parent_name iscollective       s_start ...
aper_entry             0 LimitRect      False     False None               False             0
rbend                  0 RBend           True     False None               False             0
aper_exit        6.26002 LimitRect      False     False None               False       6.26002
_end_point       6.26002                False     False None               False       6.26002

In [36]:
line2.get_aperture_table()

Compiling ContextCpu kernels...


ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored
ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored


Done compiling ContextCpu kernels.


Table: 4 rows, 10 cols
name                   s    x_aper_low   x_aper_high x_aper_low_discrete x_aper_high_discrete ...
aper_entry             0       -0.0715        0.0805             -0.0715               0.0805
rbend                  0       -0.0715        0.0805                 nan                  nan
aper_exit        6.26002       -0.0715        0.0805             -0.0715               0.0805
_end_point       6.26002       -0.0715        0.0805                 nan                  nan

In [46]:
aper_calc = apertls.ApertureCalculator(line2)

In [49]:
x_extent = aper_calc.compute_x_extent()

In [50]:
x_extent

array([[-0.0716,  0.0804],
       [-0.0716,  0.0804]])

In [ ]:
plt.plot()

In [41]:
line2['rbend'].name_associated_aperture = 'aper_entry'

In [ ]:
line2['rbend']

'aper_entry'

In [45]:
line2.slice_thick_elements(
    slicing_strategies=[
        # Slicing with thin elements
        xt.Strategy(slicing=xt.Teapot(10))])

The line already has an associated tracker


Slicing line:   0%|          | 0/3 [00:00<?, ?it/s]

{'aper_entry': ['aper_entry'],
 'rbend': ['rbend_entry',
  'rbend_aper..0',
  'rbend..entry_map',
  'drift_rbend..0',
  'rbend_aper..1',
  'rbend..0',
  'drift_rbend..1',
  'rbend_aper..2',
  'rbend..1',
  'drift_rbend..2',
  'rbend_aper..3',
  'rbend..2',
  'drift_rbend..3',
  'rbend_aper..4',
  'rbend..3',
  'drift_rbend..4',
  'rbend_aper..5',
  'rbend..4',
  'drift_rbend..5',
  'rbend_aper..6',
  'rbend..5',
  'drift_rbend..6',
  'rbend_aper..7',
  'rbend..6',
  'drift_rbend..7',
  'rbend_aper..8',
  'rbend..7',
  'drift_rbend..8',
  'rbend_aper..9',
  'rbend..8',
  'drift_rbend..9',
  'rbend_aper..10',
  'rbend..9',
  'drift_rbend..10',
  'rbend_aper..11',
  'rbend..exit_map',
  'rbend_exit'],
 'aper_exit': ['aper_exit']}

In [51]:
tt = line2.get_table()

In [52]:
tt

Table: 40 rows, 11 cols
name                         s element_type        isthick isreplica parent_name iscollective ...
aper_entry                   0 LimitRect             False     False None               False
rbend_entry                  0 Marker                False     False None               False
rbend_aper..0                0 LimitRect             False      True aper_entry         False
rbend..entry_map             0 ThinSliceRBendEntry   False     False rbend              False
drift_rbend..0               0 DriftSliceRBend        True     False rbend              False
rbend_aper..1         0.284546 LimitRect             False      True aper_entry         False
rbend..0              0.284546 ThinSliceRBend        False     False rbend              False
drift_rbend..1        0.284546 DriftSliceRBend        True     False rbend              False
rbend_aper..2         0.916871 LimitRect             False      True aper_entry         False
rbend..1              0.916871 T

In [53]:
aper_calc = apertls.ApertureCalculator(line2)
x_extent = aper_calc.compute_x_extent()

In [54]:
x_extent

array([[-0.0716,  0.0804],
       [-0.0716,  0.0804]])